En este notebook vemos cómo usar los `modelos entrenados` que produce la clase scorecard para hacer predicciones en dos datasets de Kaggle 😎

<span style='color:blue'>Importamos los módulos

In [1]:
import sys, numpy as np, pandas as pd, memento as me

<span style='color:blue'>Cargamos el primer dataset, del **Titanic**

In [2]:
df_train = pd.read_csv('train titanic.csv')
L = list(me.proc_freq(df_train, 'Survived').frequency)
print('Sobrevivieron {}. Murieron {} (entre ellos Leonardo DiCaprio)'.format(L[1], L[0]))
X, y = df_train.drop('Survived', axis=1), df_train.Survived.values

Sobrevivieron 342. Murieron 549 (entre ellos Leonardo DiCaprio)


<span style='color:blue'>Sacamos una primera scorecard excluyendo las variables `PassengerId` y `Name`

In [3]:
modelo_titanic1 = me.scorecard(excluded_vars=['PassengerId', 'Name']).fit(X, y)

Particionado 70-30 estratificado en el target terminado
------------------------------------------------------------------------------------------------------------------------
Autogrouping terminado. Máximo número de buckets = 5. Mínimo porcentaje por bucket = 0.05
------------------------------------------------------------------------------------------------------------------------
Cuidado, has puesto un valor numero máximo de iteraciones (14) superior al número de variables candidatas (9)
------------------------------------------------------------------------------------------------------------------------
Posibles variables con overfitting: ['Ticket']
------------------------------------------------------------------------------------------------------------------------
Step 01 | 0:00:00.172926 | pv = 8.59e-39 | Gini train = 55.81% | Gini test = 47.71% ---> Feature selected: Sex
Step 02 | 0:00:00.296943 | pv = 8.16e-18 | Gini train = 97.35% | Gini test = 45.15% ---> Feature selec

<span style='color:blue'>Tremendo batacazo del train al test... Esto es un claro ejemplo de **sobreajuste**, de hecho, el propio log del modelo nos avisa de que cuidado con `Ticket`. Vemos que tiene sentido: es una variable con un iv desmesuradamente alto y con muchísima granularidad

In [4]:
display(modelo_titanic1.tabla_ivs.head(3))
print('Valores distintos variable Ticket: {}'.format(len(X['Ticket'].value_counts())))

,variable,iv
0,Ticket,7.014385
1,Cabin,1.529741
2,Sex,1.479819


Valores distintos variable Ticket: 681


<span style='color:blue'>Sacamos una segunda versión del modelo excluyendo esta variable `Ticket`

In [5]:
modelo_titanic2 = me.scorecard(excluded_vars=['PassengerId', 'Name', 'Ticket']).fit(X, y)

Particionado 70-30 estratificado en el target terminado
------------------------------------------------------------------------------------------------------------------------
Autogrouping terminado. Máximo número de buckets = 5. Mínimo porcentaje por bucket = 0.05
------------------------------------------------------------------------------------------------------------------------
Cuidado, has puesto un valor numero máximo de iteraciones (14) superior al número de variables candidatas (8)
------------------------------------------------------------------------------------------------------------------------
Step 01 | 0:00:00.206477 | pv = 8.59e-39 | Gini train = 55.81% | Gini test = 47.71% ---> Feature selected: Sex
Step 02 | 0:00:00.278350 | pv = 1.08e-11 | Gini train = 67.27% | Gini test = 64.78% ---> Feature selected: Pclass
Step 03 | 0:00:00.241625 | pv = 3.03e-08 | Gini train = 72.65% | Gini test = 61.45% ---> Feature selected: Cabin
Step 04 | 0:00:00.248418 | pv = 2.48e-04 | 

<span style='color:blue'>Mucho mejor, sin embargo, seguimos teniendo cierto desplome

In [6]:
print('Caida de Gini de train a test: {:.2f}%'.format(
100*(modelo_titanic2.gini_train - modelo_titanic2.gini_test) / modelo_titanic2.gini_train))

Caida de Gini de train a test: 18.44%


<span style='color:blue'>Vemos que la diferencia entre tain y test se acentúa tras añadir la variable `Cabin`... Sacamos una tercera versión excluyendo también a `Cabin`

In [7]:
modelo_titanic3 = me.scorecard(excluded_vars=['PassengerId', 'Name', 'Ticket', 'Cabin']).fit(X, y)

Particionado 70-30 estratificado en el target terminado
------------------------------------------------------------------------------------------------------------------------
Autogrouping terminado. Máximo número de buckets = 5. Mínimo porcentaje por bucket = 0.05
------------------------------------------------------------------------------------------------------------------------
Cuidado, has puesto un valor numero máximo de iteraciones (14) superior al número de variables candidatas (7)
------------------------------------------------------------------------------------------------------------------------
Step 01 | 0:00:00.199465 | pv = 8.59e-39 | Gini train = 55.81% | Gini test = 47.71% ---> Feature selected: Sex
Step 02 | 0:00:00.202337 | pv = 1.08e-11 | Gini train = 67.27% | Gini test = 64.78% ---> Feature selected: Pclass
Step 03 | 0:00:00.210616 | pv = 5.87e-06 | Gini train = 71.36% | Gini test = 65.31% ---> Feature selected: Age
---------------------------------------------

<span style='color:blue'>Este último modelo solo tiene 3 variables.. Bueno y qué pasa?! `#minimalism` 😊

In [8]:
me.pretty_scorecard(modelo_titanic3, 'green')

,Variable,Group,Count,Percent,Goods,Bads,Bad rate,WoE,IV,Raw score,Aligned score
0,Sex,['female'],221,0.354735,54,167,0.755656,-1.603189,0.894771,1.643224,124
1,Sex,['male'],402,0.645265,330,72,0.179104,1.048248,0.585048,-1.074424,202
2,Pclass,"(-inf, 1.50)",140,0.224719,55,85,0.607143,-0.909497,0.193195,1.000029,143
3,Pclass,"[1.50, 2.50)",133,0.213483,66,67,0.503759,-0.489217,0.053060,0.537914,156
4,Pclass,"[2.50, inf)",350,0.561798,263,87,0.248571,0.632067,0.202817,-0.694983,191
5,Age,Missing,122,0.195827,85,37,0.303279,0.357554,0.023793,-0.394637,183
6,Age,"(-inf, 6.50)",35,0.056180,10,25,0.714286,-1.390470,0.109236,1.534678,127
7,Age,"[6.50, 17.50)",43,0.069021,24,19,0.441860,-0.240564,0.004089,0.265513,164
8,Age,"[17.50, 21.50)",75,0.120385,56,19,0.253333,0.606734,0.040248,-0.669659,191
9,Age,"[21.50, 50.50)",307,0.492777,181,126,0.410423,-0.111964,0.006252,0.123576,168


<span style='color:blue'>Vamos a usar estos modelos para hacer **predicciones** y subir las submissions resultantes a Kaggle

In [9]:
df_test = pd.read_csv('test titanic.csv')

<span style='color:blue'>El método `.predict()` es el responsable de aplicar el modelo a un nuevo dataframe. La variable que contiene la puntuación final es `scorecardpoints`, esta puntuación está calibrada con PEO (points equal odds, donde la tasa de malos es 1/2) en 500 puntos y un PDO (points to double odds) de 20 puntos. También se genera, para cada variable del modelo final, una columna con la puntuación parcial que aporta dicha variable (las columnas 'scr_'). Por último, podemos darle un umbral para sacar la columna 'prediction' con una predicción binaria

In [10]:
modelo_titanic1.predict(df_test, keep_columns=['PassengerId'], binary_treshold=500)

,PassengerId,Sex,Ticket,SibSp,Cabin,Pclass,scr_Sex,scr_Ticket,scr_SibSp,scr_Cabin,scr_Pclass,scorecardpoints,prediction
0,892,male,330911,0,NaN,3,145,3,95,112,81,436,1
1,893,female,363272,1,NaN,3,38,3,134,112,81,368,1
2,894,male,240276,0,NaN,2,145,3,95,112,120,475,1
3,895,male,315154,0,NaN,3,145,3,95,112,81,436,1
4,896,female,3101298,1,NaN,3,38,3,134,112,81,368,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,male,A.5. 3236,0,NaN,3,145,3,95,112,81,436,1
414,1306,female,PC 17758,0,C105,1,38,271,95,14,134,552,0
415,1307,male,SOTON/O.Q. 3101262,0,NaN,3,145,3,95,112,81,436,1
416,1308,male,359309,0,NaN,3,145,3,95,112,81,436,1


<span style='color:blue'>Generamos las submissions como archivos .csv para subirlas a Kaggle

In [11]:
modelo_titanic1.predict(df_test, keep_columns=['PassengerId'], binary_treshold=500)\
.rename(columns={'prediction': 'Survived'})[['PassengerId', 'Survived']]\
.to_csv('submission_titanic1.csv', index=False) # Score en Kaggle = 0.44258

modelo_titanic2.predict(df_test, keep_columns=['PassengerId'], binary_treshold=500)\
.rename(columns={'prediction': 'Survived'})[['PassengerId', 'Survived']]\
.to_csv('submission_titanic2.csv', index=False) # Score en Kaggle = 0.72727

modelo_titanic3.predict(df_test, keep_columns=['PassengerId'], binary_treshold=500)\
.rename(columns={'prediction': 'Survived'})[['PassengerId', 'Survived']]\
.to_csv('submission_titanic3.csv', index=False) # Score en Kaggle = 0.77272

<span style='color:blue'>Vamos ahora con otro dataframe de Kaggle similar: **Spaceship Titanic**

In [12]:
df_train = pd.read_csv('train dimension.csv')
df_train['Transported'] = np.where(df_train['Transported'] == True, 1, 0)
L = list(me.proc_freq(df_train, 'Transported').frequency)
print('Trasportados: {}. No trasportados: {}'.format(L[1], L[0]))
X, y = df_train.drop('Transported', axis=1), df_train.Transported.values

Trasportados: 4378. No trasportados: 4315


<span style='color:blue'>Tiramos un primer modelo excluyendo las variables `PassengerId` y `Name`

In [13]:
modelo_spaceship1 = me.scorecard(excluded_vars=['PassengerId', 'Name']).fit(X, y)

Particionado 70-30 estratificado en el target terminado
------------------------------------------------------------------------------------------------------------------------
Autogrouping terminado. Máximo número de buckets = 5. Mínimo porcentaje por bucket = 0.05
------------------------------------------------------------------------------------------------------------------------
Variables no agrupadas por algún error, seguramente por excesiva concentración en algún valor (> 95%) : ['VIP']
------------------------------------------------------------------------------------------------------------------------
Cuidado, has puesto un valor numero máximo de iteraciones (14) superior al número de variables candidatas (10)
------------------------------------------------------------------------------------------------------------------------
Posibles variables con overfitting: ['Cabin']
-----------------------------------------------------------------------------------------------------

<span style='color:blue'>Hacemos tres observaciones:

1) <span style='color:blue'>La primera variable que está entrando, `CryoSleep`, es booleana, no hay ningún problema

In [14]:
me.proc_freq(X, 'CryoSleep')

,frequency,percent
CryoSleep,,
False,5439,0.625676
True,3037,0.349362
NaN,217,0.024963


2) <span style='color:blue'>Nos avisa de que la variable `VIP` no ha podido agruparse por excesiva concentración y resulta que efectivamente hay un valor que se repite en más del 95% de los casos (podría ser que no fuera superior al 95% en la muestra total y si lo fuera la submuestra del train).<br>Si quisiéramos que el modelo intentará incluir esta variable habría que pedir más flexibilidad en el autogrouping rebajando el umbral mínimo de porcentaje de población por grupo, por ejemplo pasando como argumento a scorecard autogrp_dict_min_pct={'VIP': 0.02})

In [15]:
me.proc_freq(X, 'VIP')

,frequency,percent
VIP,,
False,8291,0.953756
True,199,0.022892
NaN,203,0.023352


3) <span style='color:blue'>Al igual que antes nos avisa de posible sobreajuste por `Cabin`, probamos a quitarla

In [16]:
modelo_spaceship2 = me.scorecard(excluded_vars=['PassengerId', 'Name', 'Cabin']).fit(X, y)

Particionado 70-30 estratificado en el target terminado
------------------------------------------------------------------------------------------------------------------------
Autogrouping terminado. Máximo número de buckets = 5. Mínimo porcentaje por bucket = 0.05
------------------------------------------------------------------------------------------------------------------------
Variables no agrupadas por algún error, seguramente por excesiva concentración en algún valor (> 95%) : ['VIP']
------------------------------------------------------------------------------------------------------------------------
Cuidado, has puesto un valor numero máximo de iteraciones (14) superior al número de variables candidatas (9)
------------------------------------------------------------------------------------------------------------------------
Step 01 | 0:00:00.628840 | pv = 0.00e+00 | Gini train = 43.60% | Gini test = 44.52% ---> Feature selected: CryoSleep
Step 02 | 0:00:00.692885 | pv =

<span style='color:blue'>En esta segunda versión ya no teneos sobreajuste ninguno... Sin embargo, podemos echar un vistazo a los p-valores a ver

In [17]:
modelo_spaceship2.pvalues

{'CryoSleep': 8.501261536649443e-06,
 'RoomService': 3.7894758380443915e-58,
 'Spa': 1.3662015415267725e-64,
 'HomePlanet': 6.614309530531033e-79,
 'VRDeck': 9.284706908288542e-58,
 'Destination': 6.749699441276009e-07}

<span style='color:blue'>Claramente, `CryoSleep` y `Destination` tienen unos pvalores que, si bien sí son superiores al umbral por defecto de 0.01 (porque si no habrían salido en algún paso del stepwise), son muy inferiores a la media del resto de features...<br>Podemos mirar si prescindiendo de alguna de ellas (o de ambas) obtenemos un modelo con métricas similares tanto en train como en test

In [18]:
features1 = [i for i in modelo_spaceship2.features if i != 'CryoSleep']
# features2 = [i for i in modelo_spaceship2.features if i != 'Destination']
# features3 = [i for i in modelo_spaceship2.features if i != 'CryoSleep' and i!= 'Destination']

modelo_spaceship2_alt1 = me.scorecard(features=features1).fit(X, y)
# modelo_spaceship2_alt2 = me.scorecard(features=features2).fit(X, y)
# modelo_spaceship2_alt3 = me.scorecard(features=features3).fit(X, y)

Particionado 70-30 estratificado en el target terminado
------------------------------------------------------------------------------------------------------------------------
Autogrouping terminado. Máximo número de buckets = 5. Mínimo porcentaje por bucket = 0.05
------------------------------------------------------------------------------------------------------------------------
Step 01 | 0:00:00.000000 | pv = 0.00e+00 | Gini train = 36.75% | Gini test = 35.26% ---> Feature selected: RoomService
Step 02 | 0:00:00.000000 | pv = 0.00e+00 | Gini train = 53.76% | Gini test = 53.85% ---> Feature selected: Spa
Step 03 | 0:00:00.000000 | pv = 4.76e-76 | Gini train = 61.45% | Gini test = 62.50% ---> Feature selected: HomePlanet
Step 04 | 0:00:00.000000 | pv = 1.63e-83 | Gini train = 66.82% | Gini test = 67.51% ---> Feature selected: VRDeck
Step 05 | 0:00:00.000000 | pv = 4.27e-07 | Gini train = 67.53% | Gini test = 68.35% ---> Feature selected: Destination
-------------------------------

<span style='color:blue'>En las tres pruebas obtenemos Ginis muy similares... Dejamos como opción 3 la que elimina solo al `CryoSleep` ya que da un Gini ligeramente superior tanto en train como en test respecto a lo que teníamos. Al final cuánto más sencillo sea un modelo, mejor. `#make_it_simple`

In [19]:
modelo_spaceship3 = modelo_spaceship2_alt1

<span style='color:blue'>Hacemos las predicciones y subimos los resultados a Kaggle 🚀

In [20]:
df_test = pd.read_csv('test dimension.csv')

prediction1 = modelo_spaceship1.predict(df_test, keep_columns=['PassengerId'], binary_treshold=500)\
.rename(columns={'prediction': 'Transported'})[['PassengerId', 'Transported']]
prediction1['Transported'] = np.where(prediction1['Transported'] == 1, True, False)
prediction1.to_csv('submission_spaceship1.csv', index=False) # Score en Kaggle = 0.51718

prediction2 = modelo_spaceship2.predict(df_test, keep_columns=['PassengerId'], binary_treshold=500)\
.rename(columns={'prediction': 'Transported'})[['PassengerId', 'Transported']]
prediction2['Transported'] = np.where(prediction2['Transported'] == 1, True, False)
prediction2.to_csv('submission_spaceship2.csv', index=False) # Score en Kaggle = 0.77788

prediction3 = modelo_spaceship3.predict(df_test, keep_columns=['PassengerId'], binary_treshold=500)\
.rename(columns={'prediction': 'Transported'})[['PassengerId', 'Transported']]
prediction3['Transported'] = np.where(prediction3['Transported'] == 1, True, False)
prediction3.to_csv('submission_spaceship3.csv', index=False) # Score en Kaggle = 0.77788

<span style='color:blue'>Mostramos la scorecard del último modelo, que desempeña igual que el anterior y es más sencillo

In [21]:
me.pretty_scorecard(modelo_spaceship3, color1='yellow')

,Variable,Group,Count,Percent,Goods,Bads,Bad rate,WoE,IV,Raw score,Aligned score
0,RoomService,Missing,126,0.020707,73,53,0.420635,0.334958,0.002305,-0.262972,107
1,RoomService,"(-inf, 0.50)",3870,0.635990,1408,2462,0.636176,-0.544013,0.183353,0.427099,87
2,RoomService,"[0.50, 29.50)",462,0.075924,309,153,0.331169,0.717694,0.037607,-0.563455,115
3,RoomService,"[29.50, 353.50)",617,0.101397,386,231,0.374392,0.528210,0.027703,-0.414693,111
4,RoomService,"[353.50, 729.50)",382,0.062777,300,82,0.214660,1.311854,0.095220,-1.029924,129
5,RoomService,"[729.50, inf)",628,0.103205,544,84,0.133758,1.882923,0.287572,-1.478265,142
6,Spa,Missing,124,0.020378,60,64,0.516129,-0.049748,0.000050,0.043376,98
7,Spa,"(-inf, 0.50)",3737,0.614133,1363,2374,0.635269,-0.540097,0.174574,0.470925,86
8,Spa,"[0.50, 8.50)",344,0.056532,200,144,0.418605,0.343295,0.006606,-0.299328,108
9,Spa,"[8.50, 266.50)",832,0.136730,533,299,0.359375,0.592869,0.046799,-0.516938,114
